# 1 Collaborative filtering
## 1.1 Introduction with small fictional database

The first thing we want to explore are the possibilities to perform collaborative filtering using singular value decomposition (SVD). We want to do this using the lenskit python-module. Before we apply this module to the wine dataset, we first want to get a better feeling on how this module works. To do so, we first use a small fictional database. The small database is loaded into memory as a pandas dataframe in the cell below.

**Important**: Lenskit works only if the version of the python interpreter is >= 3.8 and < 3.12 

In [40]:
import pandas as pd
fictional_ratings_df = pd.DataFrame({
    'user': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7],
    'item': [1, 2, 3, 1, 2, 4, 2, 3, 5, 1, 4, 5, 2, 3, 6, 1, 5, 6, 3, 5, 6],
    'rating': [5, 3, 4, 4, 5, 2, 3, 4, 5, 3, 2, 4, 5, 4, 3, 4, 5, 3, 4, 5, 4]
})

If we want to apply SVD to this database, this dataframa has to be compressed to a sparse row matrix, or CSR-matrix for short. The SVD-models do this automatically, but for the sake of completeness, the CSR-matrix is shown below. Missing values are filled with zeros.

In [41]:
fictional_ratings_matrix = fictional_ratings_df.pivot(index='user', columns='item', values='rating').fillna(0)

print(fictional_ratings_matrix)


item    1    2    3    4    5    6
user                              
1     5.0  3.0  4.0  0.0  0.0  0.0
2     4.0  5.0  0.0  2.0  0.0  0.0
3     0.0  3.0  4.0  0.0  5.0  0.0
4     3.0  0.0  0.0  2.0  4.0  0.0
5     0.0  5.0  4.0  0.0  0.0  3.0
6     4.0  0.0  0.0  0.0  5.0  3.0
7     0.0  0.0  4.0  0.0  5.0  4.0


## 1.2 sklearn.decomposition.TruncatedSVD
Lenskit offers only one way to perform SVD for collaborative filtering, namely via the BiasedSVD class. This class builds upon the TruncatedSVD class in sklearn.decomposition module. So before we use BiasedSVD, we first have to understand TruncatedSVD.

The idea behind TruncatedSVD is fairly simple. To start, the CSR-matrix is directly decomposed into a $U, \Sigma$ and $V^T$ matrix. This is contrary to usual SVD, where the mean value is subtracted from the values in te CSR-matrix before the SVD is performed. So if the original fictional ratings csr matrix is denoted with A, TruncatedSVD first performs the following step

$A = U \Sigma V^T$

After the SVD, only the k most relevant latent features (or singular values) are kept. How many relevant features the model will used, must be given by the user in the construction of the svd model via the n_components parameter. This approximation boils down to the the following:

$A \approx A_k = U_k \Sigma_k V_{k}^{T}$

The function svd.fit_transform returns the matrix $U_k \Sigma_k$  

In [42]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2)
fic_rat_mat_reduced = svd.fit_transform(fictional_ratings_matrix)

print("\nReduced User-Item Matrix:")
print(fic_rat_mat_reduced)


Reduced User-Item Matrix:
[[ 5.30915074 -3.65414172]
 [ 3.97486418 -4.47356679]
 [ 6.03087294  1.09853806]
 [ 3.60842275  1.67193835]
 [ 5.03382385 -2.8317348 ]
 [ 5.30668007  2.91275633]
 [ 5.93084204  3.93222308]]


To get the predicted values for the rating, we have to multiply the matrix $U_k \Sigma_k$ with the matrix $V_{k}^{T}$. This multiplication is performed by calling the inverse_transform method of the TruncatedSVD object

In [43]:
import numpy as np
fic_rat_mat_approx = svd.inverse_transform(fic_rat_mat_reduced)

print("\nReconstructed User-Item Matrix (approximation):")
print(np.round(fic_rat_mat_approx, 2)) # Rounded for readability


Reconstructed User-Item Matrix (approximation):
[[ 3.2   4.63  2.91  0.74  0.49  0.75]
 [ 2.88  4.58  2.32  0.69 -0.79  0.16]
 [ 2.16  1.92  2.86  0.41  4.07  2.07]
 [ 1.01  0.5   1.63  0.17  3.12  1.47]
 [ 2.86  3.99  2.7   0.65  0.89  0.86]
 [ 1.35  0.45  2.36  0.21  4.89  2.27]
 [ 1.32  0.07  2.58  0.18  5.91  2.69]]


## 1.3 BiasedSVD
In BiasedSVD, three biases are used
* $\mu$:   General mean
* $b_u$:   Mean of all items per user (some users give structural higher/ lower ratings than others)
* $b_i$:   Mean of all users per item (some items are structurally higher/ lower rated than others)

BiasedSVD assumes the following model
$A_{ui,real} = \mu + b_u + b_i + A_{ui,pure}$

BiasedSVD works according to the steps below:
1. First a csr matrix is formed from the dataframe. 
2. Secondly, all the means are subtracted from the csr matrix. 
3. Then, it decomposes the new csr matrix. The results of this decomposition are saved in the attributes of the BiasedSVD object
4. After this, the relevant rows and vectors are multiplied to get the relevant $A_{ui,pure}$. Thereafter, the biases are added to this value.

Steps 1 until 3 are performed in the BiasedSVD.fit method. Step 4 is executed in the BiasedSVD.predict_for_user method. 

It is important to note that, contrary to TruncatedSVD, there is no way to compute the whole prediction matrix at once using the BiasedSVD class

In [45]:
from lenskit.algorithms.svd import BiasedSVD

biased_svd = BiasedSVD(features=2)  # 2 latent features

biased_svd.fit(fictional_ratings_df)

user_id = 2
all_items = fictional_ratings_df['item'].unique()
b_svd_recommendations = biased_svd.predict_for_user(user_id, all_items)
print(f'Recommendations for user {user_id}:', b_svd_recommendations)

Recommendations for user 2: 1    3.496016
2    4.902816
3    3.884709
4    2.106290
5    4.310345
6    3.515951
dtype: float64


## 1.4 FunkSVD
Asides from the BiasedSVD class does the lenskit module contain another class that contains the acronym SVD in its name: FunkSVD. FunkSVD is named after Simon Funk who popularized it during the Netflix Prize competition. 

The FunkSVD algorithm wants to decompose the ratingsmatrix A in two matrices, a U matrix representing the users and a V matrix representing the items, similar to a regular singular value decomposition. In math:

$A = UV^T$

However, the FunkSVD algorithm does not generate directly the U and V matrices. Instead, it calculates them row and column-wise using stochastic gradient descent (SGD). The here implemented SGD makes use of a regularization factor to prevent the entries in the U and V matrices from getting too large values. For that reason, the optimization problem looks like this:

$\min \sum_{(u,i) \in \text{ratings}} (A_{ui} - \hat{A}_{ui})^2 + \lambda (\|U_u\|^2 + \|V_i\|^2)$

In this equation
* $A_{ui}$ stands for the real rating
* $\hat{A}_{ui}$ stands for the calculated rating
* $U_u$ stands for the row representing user u in the U matrix
* $V_i$ stands for the row representing user i in the V matrix
* $\lambda$ stands for the regularization penalty

The number of iterations, the learning rate and regularization penalty can be specified when constructing the FunkSVD object. Furthermore, the "number of features" of the decomposition can also be passed in this constructor, just as was the case in the previous two SVDs.

All of the above is performed by the FunkSVD.fit method. The FunkSVD.predict_for_user adds bias to the results for the model obtained by the fit method. Just like with BiasedSVD, FunkSVD can only make predictions for one user at a time.

As a side note, this implementation of FunkSVD uses only one rating per iteration, making it a "real" SGD and not a batch process.  

In [50]:
from lenskit.algorithms.funksvd import FunkSVD

funk_svd = FunkSVD(features=2, iterations=100, lrate=0.01, reg=0.1)
funk_svd.fit(fictional_ratings_df)

user_id = 2
all_items = fictional_ratings_df['item'].unique()
f_svd_recommendations = funk_svd.predict_for_user(user_id, all_items)
print(f'Recommendations for user {user_id}:', f_svd_recommendations)

Recommendations for user 2: 1    3.947330
2    3.939892
3    3.923106
4    3.270567
5    4.321505
6    3.623049
dtype: float64


## 1.5 BiasedMF
One last decomposition algorithm from the lenskit module to consider is called BiasedMF (biased matrix factorization). BiasedMF wants to split the ratings matrix A in a U and V matrix, comparable to the FunkSVD. Also similar to FunkSVD, it does this using an optimization problem. However, where FunkSVD optimizes for one rating at the time, BiasedMF optimizes the whole U and V matrix at once. It does this using a technique called alternating least squares (ALS). In this technique, in one iteration, the problem is optimized for U, in the next iteration, the problem is optimized for V. 

The corresponding optimization problem can be stated as follows:

$\min \sum_{(u,i) \in A} (A_{ui} - \hat{A}_{ui})^2 + \lambda \left( \|U\|^2 + \|V\|^2 + b_u^2 + b_i^2 \right)$ 

In [51]:
from lenskit.algorithms.als import BiasedMF

biased_mf = BiasedMF(features=2, iterations=20, reg=0.1)

biased_mf.fit(fictional_ratings_df)

b_mf_recommendations = biased_mf.predict_for_user(user_id, all_items)
print(f'Recommendations for user {user_id}:', b_mf_recommendations)


Recommendations for user 2: 1    3.739562
2    4.839057
3    3.872617
4    2.164538
5    3.486846
6    3.727821
dtype: float64
